# Williams_6835_210227A5 - rna_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [3]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/raw_reads
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/processed_raw_reads
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/jsons
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs

Save metadata file

In [4]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata/rna_seq_download_metadata.Williams_6835_210227A5.txt
Sequencing core user	Sequencing core project	Sequencing core password	Sequencing core library name	Name	Paired-end or single-end	Genome	Library type	Umis	Control	Strand specificity	Blacklist removal	With ercc spike-in
gdj5	Williams_6835_210227A5	XXXXXXX	Isoprenaline_0_1	0.1uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj5	Williams_6835_210227A5	XXXXXXX	Isoprenaline_0_5	0.5uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj5	Williams_6835_210227A5	XXXXXXX	Isoprenaline_1_0	1.0uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj5	Williams_6835_210227A5	XXXXXXX	Isoprenaline_5_0	5.0uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj5	Williams_6835_210227A5	XXXXXXX	Isoprenaline_10_0	10.0uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj6	Williams_6835_210227A5	XXXXXXX	Isoprenaline_50_0	50.0uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj7	Williams_6835_210227A5	XXXXXXX	Isoprenaline_100_0	100.0uM_Isoprenaline_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False
gdj8	Williams_6835_210227A5	XXXXXXX	Isoprenaline_DMSO	DMSO_IsoprenalineDoseResponse	PE	hg38	RNA-seq	False	DMSO_IsoprenalineDoseResponse	revstranded	True	False


Overwriting /data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata/rna_seq_download_metadata.Williams_6835_210227A5.txt


#### Download FASTQ from dukeds
Create file to download FASTQ files

In [5]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/download_Williams_6835_210227A5.sh
#!/bin/bash
METADATA=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata/rna_seq_download_metadata.Williams_6835_210227A5.txt
DATA_HOME=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq
mkdir -p ${DATA_HOME}/raw_reads/

module load ddsclient
ddsclient download -p Williams_6835_210227A5 ${DATA_HOME}/raw_reads/Williams_6835_210227A5


Writing /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/download_Williams_6835_210227A5.sh


Execute file to download files

In [6]:
%%script --out blocking_job_str bash
sbatch -o /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs/Williams_6835_210227A5_download_fastq_files.out \
 -p all,new \
 --wrap="ssh aeb84@Hardac-xfer.genome.duke.edu 'sh /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/download_Williams_6835_210227A5.sh' "

Extract blocking job id

In [7]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Merge lanes of FASTQ files

In [8]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/merge_lanes_Williams_6835_210227A5.sh
#!/bin/bash
#SBATCH --array=0-8%20
ORDER=Williams_6835_210227A5
RAW_DATA_DIR=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/raw_reads/${ORDER}
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata/rna_seq_download_metadata.Williams_6835_210227A5.txt

mkdir -p ${PROCESSED_DATA_DIR}
cd ${PROCESSED_DATA_DIR}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}"
    exit 1
fi

name_header=$(/bin/grep -Poi "\tname\t" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Library Name column not found in ${METADATA}"
    exit 1
fi
name_header=$(echo ${name_header} | cut -f2)

seq_type_header=$(head -1 ${METADATA} | /bin/grep -Poi "paired.?end.?or.?single.?end")
if [[ $? == 1 ]];
then
    echo -e "ERROR: Paired-end or single-end column not found in ${METADATA}"
    exit 1
fi

sample_seq_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
sample_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
seq_type=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_type_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');


for read_pair in R1 R2 UMI;
do
    sample_files=$(/bin/ls ${RAW_DATA_DIR}/${sample_seq_name/ /}_S[0-9]*_L[0-9][0-9][0-9]_${read_pair}_* 2> /dev/null)
    if [[ $? != 0 ]]; # If no samples found with that read_pair, continue
    then
        continue;
    fi
    if [[ ${read_pair} == "R1" || (${seq_type/ /} == "PE" || ${seq_type/ /} == "pe") ]];
    then
        # Merge all lanes
        merged=$(basename $(echo ${sample_files} | awk '{print $1}') | sed -e 's/_L[0-9]\{3\}_/_/')
        cat ${sample_files} > ${merged};

        # Rename samples with our sample Names
        dest_filename=$(basename $(echo ${merged} | awk '{print $1}') | sed -r 's/\_S[0-9]+//; s/\_(R1|R2|UMI)\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
        mv ${merged} ${dest_filename}

        cleaned_dest_filename=${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

        if [[ ${seq_type/ /} == "SE" || ${seq_type/ /} == "se" ]];
        then
            cleaned_dest_filename=${cleaned_dest_filename/.R1/}
        fi
        
        mv ${dest_filename} ${cleaned_dest_filename}
    fi
done


Writing /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/merge_lanes_Williams_6835_210227A5.sh


Execute file to merge lanes of FASTQ files

In [9]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs/Williams_6835_210227A5_merge_fastq_files_%a.out \
 -p all,new \
 --depend afterok:$1 \
 --array 0-7%20 \
 /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/merge_lanes_Williams_6835_210227A5.sh

Extract blocking job id

In [10]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create JSON files for CWL pipeline files

In [11]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/cwl_json_gen_Williams_6835_210227A5.sh
#!/bin/bash
ORDER=Williams_6835_210227A5
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20210312_Graham//data/rna_seq/metadata/rna_seq_download_metadata.Williams_6835_210227A5.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/jsons \
    -t rna-seq \
    --fastq-gzipped \
    --mem 48000 \
    --nthreads 24 \
    --separate-jsons \
    --skip-star-2pass \


Writing /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/cwl_json_gen_Williams_6835_210227A5.sh


Execute file to create JSON files

In [12]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -o /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs/Williams_6835_210227A5_cwl_json_gen.out \
 -p all,new \
 --depend afterok:$1 \
 /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/cwl_json_gen_Williams_6835_210227A5.sh

Extract blocking job id

In [13]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for pe-revstranded-with-sjdb samples

In [14]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/Williams_6835_210227A5-pe-revstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=cwl_rna_seq
#SBATCH --output=/data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs/Williams_6835_210227A5-pe-revstranded-with-sjdb-%a.out
#SBATCH --mail-user=aeb84@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=48000
#SBATCH --cpus-per-task=24

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/STAR-STAR_2.4.1a/bin/Linux_x86_64/:$PATH"
export PATH="/data/reddylab/software/subread-1.4.6-p4-Linux-x86_64/bin/:$PATH"
export PATH="/data/reddylab/software/bamtools-2.2.3/bin/:$PATH"

export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/tmpdirs/tmp-Williams_6835_210227A5-pe-revstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/tmpdirs/tmp-Williams_6835_210227A5-pe-revstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/Williams_6835_210227A5-pe-revstranded-with-sjdb  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/RNA-seq_pipeline/pipeline-pe-revstranded-with-sjdb.cwl \
    /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/jsons/rna_seq_download_metadata.Williams_6835_210227A5-pe-revstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}.json

# Delete any tmpdir not removed by cwltool
rm -rf /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/tmpdirs/tmp-Williams_6835_210227A5-pe-revstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-


Writing /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/Williams_6835_210227A5-pe-revstranded-with-sjdb.sh


Execute SLURM array master file

In [15]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all,new \
 --depend afterok:$1 \
 --array 0-7%20 \
 /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/Williams_6835_210227A5-pe-revstranded-with-sjdb.sh

Extract blocking job id

In [16]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [17]:
%%writefile /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/generate_qc_cell_Williams_6835_210227A5-pe-revstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/logs/qc_gen.Williams_6835_210227A5-pe-revstranded-with-sjdb.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/Williams_6835_210227A5-pe-revstranded-with-sjdb

python /data/reddylab/software/cwl/bin/generate_stats_rnaseq_paired_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.PBC.txt@@') \
> qc.txt

Writing /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/generate_qc_cell_Williams_6835_210227A5-pe-revstranded-with-sjdb.sh


Generate QCs for Williams_6835_210227A5-pe-revstranded-with-sjdb

In [18]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p all,new \
 --depend afterok:$1 \
 /data/reddylab/Alex/collab/20210312_Graham//processing/rna_seq/scripts/generate_qc_cell_Williams_6835_210227A5-pe-revstranded-with-sjdb.sh

Extract blocking job id

In [ ]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)